##  OSMから取得した、道路ネットワークデータを編集してみる
重み付けを行ったルーティング、道路追加、削除など

### まずは何も加工せずに、ナチュラルローソンアークヒルズ店から100円ローソン元麻布店までルーティング
黒がスタート地点、青がゴール

In [21]:
import folium
import networkx as nx
import osmnx as ox
from pyproj import Geod

ox.config(use_cache=True, log_console=True)

# 六本木交差点から半径1000ｍの道路ネットワークをOSMから生成する
ROPPONGI = (35.663236, 139.732275)
G = ox.graph_from_point(ROPPONGI, distance=3000, network_type='drive', simplify=False)

# 高速道路を除去
non_motorway = [ (u, v, k, data) for u, v, k, data in G.edges(keys=True, data=True)  if data['highway'] == 'motorway']
G.remove_edges_from(non_motorway)
G = ox.remove_isolated_nodes(G)

start_point = (35.6681770, 139.7397242)
start_node = ox.get_nearest_node(G, start_point)
end_point = (35.6578560, 139.7274620)
end_node = ox.get_nearest_node(G, end_point)

path = nx.shortest_path(G, start_node, end_node, weight='length')
map = ox.plot_route_folium(G, path, route_color='green')
folium.Marker( list(start_point), icon=folium.Icon(color='black')).add_to(map)
folium.Marker( list(end_point), icon=folium.Icon(color='blue')).add_to(map)

map


### 六本木通りを避けるように、重み付けをする

In [25]:
weights = {
            (499192175, 2590477971, 0): {'length':1000},
            (2590477971, 2138849615, 0): {'length':1000},
            (2590477966, 2590493981, 0): {'length':1000},
            (2590493981, 342022926, 0): {'length':1000}
          }

nx.set_edge_attributes(G, weights)

path = nx.shortest_path(G, start_node, end_node, weight='length')
map = ox.plot_route_folium(G, path, route_color='green')
folium.Marker( list(start_point), icon=folium.Icon(color='black')).add_to(map)
folium.Marker( list(end_point), icon=folium.Icon(color='blue')).add_to(map)

map

避けた！

### 道路を追加してみる
六本木通り交差手前のいびつな右左折をなくし、（現実ではありえないが）まっすぐ六本木通りを交差する道路を作成してみる

In [27]:
# add path from 499190948 to 1075233512
paths = (499190948, 1075233512)
G.add_path(paths)

path = nx.shortest_path(G, start_node, end_node, weight='length')
map = ox.plot_route_folium(G, path, route_color='green')
folium.Marker( list(start_point), icon=folium.Icon(color='black')).add_to(map)
folium.Marker( list(end_point), icon=folium.Icon(color='blue')).add_to(map)

map

まっすぐになった！